# ЛР 1. Непараметрическая регрессия

## Подготовка

Для начала импортируем полезные вещи для работы с массивами данных, рисования графиков и непосредственно создания моделей.

In [29]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

Загрузим датасет с цифрами. Обратите внимание на несколько вещей:

- Берётся только 10% исходного датасета для более быстрой работы;
- В этом датасете все признаки (в том числе и целевой) векторизированы за нас, а пропуски отсутствуют;

In [30]:
df = pd.read_csv('mnist_784.csv')
df = df[:700]
df.shape

(700, 785)

Разделим данные на тренировочную и тестовую выборки.

In [31]:
X = df.drop('class', axis=1)
y = df['class']

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0)

## Наивное решение

Создадим две классные штуки для решения нашей задачи:

- `KNeighborsRegressor` для окна переменного радиуса
- `RadiusNeighborsRegressor` для окна фиксированного радиуса

In [32]:
neighbours = KNeighborsRegressor()
radius = RadiusNeighborsRegressor()

Натренируем регрессор переменного радиуса:

In [34]:
params = {
    'n_neighbors': [1, 3, 5],
    'weights': [
        'uniform',
        'distance',
        lambda x: [1 - abs(u) if abs(u) < 1 else 0 for u in x],
        lambda x: [3/4 * (1 - u ** 2) if abs(u) < 1 else 0 for u in x]
    ],
    'p': [1, 2, 100000]
}

model = GridSearchCV(
    estimator=neighbours,
    param_grid=params,
    scoring='neg_mean_absolute_error',
    cv=10
)

model.fit(train_X, train_y)

AxisError: axis 1 is out of bounds for array of dimension 1